# Libraries

In [267]:
import pandas as pd
import numpy as np
import os as os
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

# Load train data

- iterate through train data folder 
- feature engineer the **country** and **league** from parent folder name
- join the loaded csv files by rows

In [268]:
df = pd.DataFrame()
for root, directory, files in os.walk("data/train", topdown=False):
    if files:
        for file in files:
            tmp = pd.read_csv(f"{root}/{file}")
            # Remove empty rows and columns
            tmp = tmp.dropna(how='all', axis=0)
            tmp = tmp.dropna(how='all', axis=1)
            # Derive additional columns
            tmp["league"] = int(root.split("\\")[2])
            tmp["country"] = root.split("\\")[1]
            tmp["season"] = int(file[:2]) # no. of season - 00/01 - 0th season, 21/22 - 21st season
            df = pd.concat([df, tmp], axis = 0)

## Data validation

In some cases, the data is wrong. This section correct the loaded data.

### Random unnamed data

File *'data/train/portugal/1/0304.csv'* contains random data in columns *'Unnamed: 33'* and *'Unnamed: 34'*.

In [269]:
unnamed_cols_df = df[['Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34']]
unnamed_cols_df[unnamed_cols_df.notnull().any(axis=1)]

,Unnamed: 32,Unnamed: 33,Unnamed: 34
157,2.0,1.8,-1.75


Since we do not know what these columns represent, and it is only one non-NA row from the whole dataset, this column is removed.

In [270]:
df = df.drop(columns = unnamed_cols_df.columns)

### Wrong betting odds names

File *'data/train/germany/2/0405.csv'* contains columns **LB**, **LB.1** and **LB.2**, which are unique only to this file. After further investigation, they represent the betting odds data for Ladbrokers. After looking at the data more thoroughly, it can be guessed that all three columns represent odds for home win, away win, and draw.

In [271]:
tmp = pd.read_csv("data/train/germany/2/0405.csv")
# Remove empty rows and columns
tmp = tmp.dropna(how='all', axis=0)
tmp = tmp.dropna(how='all', axis=1)
tmp = tmp.loc[:, ~tmp.columns.str.startswith('Unnamed:')]
tmp = tmp[tmp[['LB', 'LB.1', 'LB.2']].notnull().any(axis=1)]
tmp.filter(regex='[HDAB12]$').iloc[:, -12:]

,LB,LB.1,LB.2,SBH,SBD,SBA,WHH,WHD,WHA,GBAHH,GBAHA,GBAH
2,1.67,3.25,NaN,1.62,3.75,5.00,1.65,3.30,4.60,1.65,2.02,-0.50
6,2.00,3.20,NaN,2.10,3.20,3.25,2.15,3.10,3.00,1.77,1.95,-0.25
7,1.91,3.25,NaN,2.00,3.30,3.40,1.90,3.25,3.40,1.95,1.70,-0.50
12,2.60,3.20,2.38,2.70,3.20,2.40,2.50,3.10,2.50,1.43,2.50,0.50
15,2.25,3.20,2.75,2.40,3.00,2.88,2.20,3.10,2.87,2.30,1.48,-0.50
...,...,...,...,...,...,...,...,...,...,...,...,...
291,2.40,3.25,2.50,2.50,3.25,2.50,2.45,3.25,2.45,2.45,1.43,-0.50
294,2.00,3.00,3.40,2.10,3.20,3.25,2.10,3.20,3.00,1.91,1.70,-0.50
295,2.25,3.20,2.75,2.38,3.10,2.88,2.37,3.30,2.50,2.20,1.52,-0.50
297,1.62,3.25,5.00,1.60,3.75,5.00,1.60,3.30,5.00,1.62,2.06,-0.50


Based on column similarity, we can make an edjucated guess that

- **LB** should be **LBH**,
- **LB.1** should be **LBD**, and
- **LB.2** should be **LBA**

In [272]:
# If LB is not null, use that value and replace it in LBH
df["LBH"] = df["LBH"].mask(df["LB"  ].notnull(), df["LB"])
df["LBD"] = df["LBD"].mask(df["LB.1"].notnull(), df["LB.1"])
df["LBA"] = df["LBA"].mask(df["LB.2"].notnull(), df["LB.2"])
df = df.drop(columns = ["LB", "LB.1", "LB.2"])

### Same name for different things

In some cases, columns are named differently. We will standardize to use column names that are in *'notes.txt'*


In [277]:
for root, directory, files in os.walk("data/train", topdown=False):
    if files:
        for file in files:
            tmp = pd.read_csv(f"{root}/{file}")
            if "HT" in tmp.columns:
                print(f"Cases for HT: {root}, {file}")
            if "AT" in tmp.columns:
                print(f"Cases for AT: {root}, {file}")

Cases for HFKC: data/train\belgium\1, 1718.csv
Cases for HFKC: data/train\belgium\1, 1819.csv
Cases for HFKC: data/train\france\2, 1718.csv
Cases for HT: data/train\greece\1, 0001.csv
Cases for AT: data/train\greece\1, 0001.csv
Cases for HT: data/train\greece\1, 0102.csv
Cases for AT: data/train\greece\1, 0102.csv
Cases for HT: data/train\greece\1, 0203.csv
Cases for AT: data/train\greece\1, 0203.csv
Cases for HT: data/train\greece\1, 0304.csv
Cases for AT: data/train\greece\1, 0304.csv
Cases for HT: data/train\greece\1, 0405.csv
Cases for AT: data/train\greece\1, 0405.csv
Cases for HFKC: data/train\greece\1, 1718.csv
Cases for HFKC: data/train\greece\1, 1819.csv


The only affected data are in Greece.

In [278]:
df["HomeTeam"] = df["HomeTeam"].mask(df["HT"].notnull(), df["HT"])
df["AwayTeam"] = df["AwayTeam"].mask(df["AT"].notnull(), df["AT"])
df = df.drop(columns = ["HT", "AT"])

### !!! Free Kicks Conceded

As per the data description:

> Note that Free Kicks Conceeded (HFKC, AFKC) includes fouls, offsides and any other offense commmitted and will always be equal to or higher than the number of fouls. \
> Fouls make up the vast majority of Free Kicks Conceded. \
> Free Kicks Conceded are shown when specific data on Fouls are not available (France 2nd, Belgium 1st and Greece 1st divisions).

Which can be further verified by:

In [282]:
for root, directory, files in os.walk("data/train", topdown=False):
    if files:
        for file in files:
            tmp = pd.read_csv(f"{root}/{file}")
            if "HFKC" in tmp.columns or "AFKC" in tmp.columns:
                print(f"{root}, {file}")

data/train\belgium\1, 1718.csv
data/train\belgium\1, 1819.csv
data/train\france\2, 1718.csv
data/train\greece\1, 1718.csv
data/train\greece\1, 1819.csv


CO S TÍM DÁL AAAAA

### Negative odds

In some cases, the odds of something happening are negative.

In [306]:
numeric_columns = df.select_dtypes(exclude=object)
negative_rows = numeric_columns.loc[(numeric_columns < 0).any(axis=1), (numeric_columns < 0).any(axis=0)]
negative_rows

,FTAG,GBAHA,GBAH,B365AH,AHh,AHCh,LBAHH,LBAHA,LBAH
14,0.0,1.52,-0.5,NaN,NaN,NaN,NaN,NaN,NaN
16,1.0,2.84,-0.5,NaN,NaN,NaN,NaN,NaN,NaN
20,0.0,1.94,-0.5,NaN,NaN,NaN,NaN,NaN,NaN
22,1.0,1.73,-0.5,NaN,NaN,NaN,NaN,NaN,NaN
23,1.0,1.61,-0.5,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
366,2.0,NaN,NaN,NaN,-0.25,-0.25,NaN,NaN,NaN
367,1.0,NaN,NaN,NaN,-1.00,-1.25,NaN,NaN,NaN
368,1.0,NaN,NaN,NaN,-0.25,-0.50,NaN,NaN,NaN
370,2.0,NaN,NaN,NaN,-0.25,-0.25,NaN,NaN,NaN


There are several possible fixes:

- remove them
- transfer them to absolute values
- exponentiate them (as they could be log odds)

- - -

In [ ]:
df.shape

(154953, 215)

Drop all completely empty columns and rows which there is a lot of, dropped 41 empty columns in total.

In [ ]:
df.dropna(how='all', axis=1, inplace=True)

In [ ]:
df.shape

(155173, 174)

In [ ]:
df.to_csv("final.csv")

# Data Exploration

Total overview of all variables.

In [ ]:
df.describe()

,FTHG,FTAG,HTHG,HTAG,GBH,GBD,GBA,IWH,IWD,IWA,...,ABP,SOH,SOD,SOA,LBAHH,LBAHA,LBAH,LB,LB.1,LB.2
count,154864.000000,154841.000000,154813.000000,154820.000000,88060.000000,88123.000000,88106.000000,150729.000000,150735.000000,150766.000000,...,6671.000000,6599.000000,6613.000000,6614.000000,5470.000000,5470.000000,5458.000000,110.000000,110.000000,107.000000
mean,2.216358,1.130883,0.655636,0.494206,2.275366,3.381363,3.950892,2.335784,3.382514,3.753866,...,20.914406,2.216488,3.391382,3.833776,1.911200,1.924793,-0.350500,1.985955,3.258182,3.808477
std,21.861876,1.108967,0.809382,0.708167,1.039952,0.613622,2.195107,1.155085,0.713629,2.251540,...,17.212936,0.983779,0.481409,2.078919,0.174042,0.124442,0.588259,0.434470,0.170338,1.309792
min,0.000000,-1.000000,0.000000,0.000000,1.030000,1.250000,1.030000,1.010000,1.500000,1.030000,...,0.000000,1.060000,2.380000,1.140000,-2.250000,-0.500000,-2.250000,1.360000,3.000000,1.900000
25%,1.000000,0.000000,0.000000,0.000000,1.730000,3.100000,2.650000,1.750000,3.050000,2.550000,...,10.000000,1.700000,3.200000,2.620000,1.840000,1.840000,-0.750000,1.684250,3.200000,2.800000
50%,1.000000,1.000000,0.000000,0.000000,2.100000,3.250000,3.350000,2.100000,3.200000,3.200000,...,20.000000,2.100000,3.250000,3.250000,1.920000,1.920000,-0.250000,1.909000,3.200000,3.500000
75%,2.000000,2.000000,1.000000,1.000000,2.450000,3.450000,4.500000,2.500000,3.450000,4.200000,...,30.000000,2.400000,3.400000,4.500000,2.000000,2.000000,0.000000,2.237500,3.250000,4.333000
max,998.000000,13.000000,7.000000,6.000000,24.000000,11.000000,34.000000,30.000000,16.000000,70.000000,...,150.000000,13.000000,8.000000,21.000000,3.400000,4.000000,9.000000,3.500000,4.000000,8.000000


Percentage of missing values.

In [307]:
na_vals = df.isna().sum()
na_vals = na_vals/df.shape[0]
na_vals = na_vals.sort_values()
na_vals.sort_values(ascending = False).head(20)

HFKC          0.991572
AFKC          0.991572
SYD           0.964809
LBAH          0.964776
SYA           0.964757
SYH           0.964757
LBAHH         0.964699
LBAHA         0.964699
AO            0.958058
HHW           0.958007
AHW           0.957929
HO            0.957923
SOH           0.957413
SOD           0.957323
SOA           0.957316
ABP           0.956948
Attendance    0.956916
HBP           0.956845
B365AH        0.955735
GBAHA         0.924880
dtype: float64

List of the most missing values.

In [ ]:
na_vals = df[df["season"] > 18].isna().sum()

na_vals = na_vals/df[df["season"] > 18].shape[0]*100
na_vals

Div           0.000000
Date          0.087754
HomeTeam      0.000000
AwayTeam      0.000000
FTHG          0.034126
               ...    
LB          100.000000
LB.1        100.000000
LB.2        100.000000
HT          100.000000
AT          100.000000
Length: 174, dtype: float64